<a href="https://colab.research.google.com/github/vadhri/hpc-notebook/blob/main/finite_difference_mpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install mpi4py
!apt-get install -y openmpi-bin


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [9]:
!mpiexec --version
!echo "localhost slots=4" > my_hostfile

mpiexec (OpenRTE) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/


In [10]:
%%writefile finite-diff-2d.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

rows, cols = 10000, 15000  # Adjust as needed
np.random.seed(42)

if rank == 0:
    array_2d = np.random.rand(rows, cols) * 100
    print(f"Process 0 broadcasting array:\n{array_2d.shape}")

    # Create column splits ensuring interior points exist
    split_sizes = np.linspace(1, cols - 1, size + 1, dtype=int)  # Avoids first & last column
    subarrays = [
        array_2d[:, split_sizes[i] - 1 : split_sizes[i + 1] + 1]  # Include 1 extra column on each side
        for i in range(size)
    ]

    valid_starts = [split_sizes[i] - 1 for i in range(size)]  # Track where diffs start
    valid_col_counts = [split_sizes[i + 1] - split_sizes[i] for i in range(size)]  # Actual diff columns
    counts = np.array([subarray.size for subarray in subarrays], dtype=np.int32)
    displacements = np.insert(np.cumsum(counts), 0, 0)[:-1].astype(np.int32)
    flat_data = np.concatenate([subarray.flatten() for subarray in subarrays])
else:
    counts = displacements = flat_data = valid_col_counts = valid_starts = None

# Broadcast meta info
valid_col_counts = comm.bcast(valid_col_counts, root=0)
valid_starts = comm.bcast(valid_starts, root=0)
counts = comm.bcast(counts, root=0)
displacements = comm.bcast(displacements, root=0)

# Allocate buffer and scatter data
recv_size = counts[rank]
recv_data = np.zeros(recv_size, dtype=np.float64)
comm.Scatterv([flat_data, counts, displacements, MPI.DOUBLE], recv_data, root=0)

# Reshape to 2D
recv_cols = recv_size // rows
recv_data_2d = recv_data.reshape(rows, recv_cols)

# Debug: Check subarray received per rank
print(f"Rank {rank}: Received subarray shape = {recv_data_2d.shape}")

# Compute finite difference safely
finite_diff_cols = valid_col_counts[rank]
finite_diff = np.zeros((rows, finite_diff_cols), dtype=np.float64)

for j in range(finite_diff_cols):
    if j + 2 < recv_cols:  # Ensure index within range
        finite_diff[:, j] = (recv_data_2d[:, j + 2] - recv_data_2d[:, j]) / 2.0

# Gather results
sendbuf = finite_diff.flatten()
recvcounts = comm.gather(sendbuf.size, root=0)

if rank == 0:
    total_recv_size = sum(recvcounts)
    recvbuf = np.empty(total_recv_size, dtype=np.float64)
    displacements = np.insert(np.cumsum(recvcounts), 0, 0)[:-1].astype(np.int32)
else:
    recvbuf = None
    displacements = None

comm.Gatherv(sendbuf, [recvbuf, recvcounts, displacements, MPI.DOUBLE], root=0)

if rank == 0:
    expected_cols = cols - 2
    merged_finite_diff = np.zeros((rows, expected_cols), dtype=np.float64)
    offset = 0
    for i in range(size):
        if valid_col_counts[i] > 0:
            cols_for_rank = valid_col_counts[i]
            data = recvbuf[offset : offset + rows * cols_for_rank].reshape(rows, cols_for_rank)
            start_col = valid_starts[i]
            merged_finite_diff[:, start_col : start_col + cols_for_rank] = data
            offset += rows * cols_for_rank

    print("Merged Finite Difference Array at Process 0:", merged_finite_diff.shape)
    np.savetxt("finite_diff.txt", merged_finite_diff, fmt="%.6f")


Overwriting finite-diff-2d.py


In [11]:
!sudo mpirun  --allow-run-as-root --hostfile my_hostfile -np 4 python finite-diff-2d.py

Process 0 broadcasting array:
(10000, 15000)
Rank 1: Received subarray shape = (10000, 3752)
Rank 2: Received subarray shape = (10000, 3751)
Rank 3: Received subarray shape = (10000, 3752)
Rank 0: Received subarray shape = (10000, 3751)
Merged Finite Difference Array at Process 0: (10000, 14998)


In [15]:
import numpy as np

rows, cols = 10000, 15000  # Adjust as needed
np.random.seed(42)  # For reproducibility

array_2d = np.random.rand(rows, cols) * 100  # Random values between 0 and 100

print("Original Array:", array_2d.shape)

# Compute finite difference in the X direction (central differences only)
finite_diff_x = (array_2d[:, 2:] - array_2d[:, :-2]) / 2  # Only central cols

print("\nFinite Difference in the X Direction (Horizontal Differences):")
print(finite_diff_x.shape)

# Save the result (6 columns)
np.savetxt("finite_diff_np.txt", finite_diff_x, fmt="%.6f")


Original Array: (10000, 15000)

Finite Difference in the X Direction (Horizontal Differences):
(10000, 14998)


In [16]:
# load file into variable
finite_diff_np = np.loadtxt("finite_diff_np.txt")
finite_diff = np.loadtxt("finite_diff.txt")
finite_diff_np.shape, finite_diff.shape

((10000, 14998), (10000, 14998))

In [17]:
np.all(np.isclose(finite_diff_np, finite_diff))

True